<a href="https://colab.research.google.com/github/ibribr/DT8807/blob/master/Embedd_FNN_Umair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Convert a float to its binary representation
def float_to_binary(f):
    # Get the binary string representation (including integer and fractional parts)
    #binary_string = format(struct.unpack('!Q', struct.pack('!d', f))[0], '064b')
    binary_string = format(struct.unpack('!I', struct.pack('!f', f))[0], '032b')
    #binary_string = format(struct.unpack('!H', struct.pack('!e', f))[0], '016b')

    return binary_string

# Convert a binary string back to a float
def binary_to_float(binary_string):
    # Convert the binary string to an integer
    integer_value = int(binary_string, 2)

    # Convert the integer to a float
    #float_value = struct.unpack('!d', struct.pack('!Q', integer_value))[0]
    float_value = struct.unpack('!f', struct.pack('!I', integer_value))[0]
    #float_value = struct.unpack('!e', struct.pack('!H', integer_value))[0]

    return float_value

import struct

# Example usage
original_float = 12.5  # Replace with your float
binary_representation = float_to_binary(original_float)
print(len(binary_representation))
# LSB substot
converted_float = binary_to_float(binary_representation)

print("Original Float:", original_float)
print("Binary Representation:", binary_representation)
print("Converted Float:", converted_float)

32
Original Float: 12.5
Binary Representation: 01000001010010000000000000000000
Converted Float: 12.5


In [24]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score

# Sample data for training
data = np.array([[0,0],[0,1],[1,0],[1,1]])
labels = np.array([[0],[1],[1],[0]])

# Define a basic model
model = keras.models.Sequential([
    keras.layers.Dense(10, activation='relu', input_shape=(2,), name='hidden_layer_1'),
    keras.layers.Dense(14, activation='relu', name='hidden_layer_2'),
    keras.layers.Dense(1, activation='sigmoid', name='output_layer')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(data, labels, epochs=100, verbose=0)
y_pred = np.round(model.predict(data))
print(y_pred)
print('Model accuracy = ', accuracy_score(labels, y_pred)*100)

for i in range(len(model.layers)):
    weights = model.layers[i].get_weights()[0]     #weights
    biases = model.layers[i].get_weights()[1]        #bias
    #print(f"Layer {i} Weights: {weights}")
    #print(f"Layer {i} Biases: {biases}")
    # do something with the weights
    #print(weights.size)
    #print(weights.shape)
    for j, row in enumerate(range(weights.shape[0])):
      for k, col in enumerate(range(weights.shape[1])):
        #print(weights[j,k])
        binary_representation = float_to_binary(weights[j,k])
        #print(binary_representation)
        converted_float = binary_to_float(binary_representation)
        #print(converted_float)

1/1 [==============================] - 0s 42ms/step
[[0.]
 [1.]
 [1.]
 [0.]]
Model accuracy =  100.0


In [25]:
def message_to_binary(message):
    return ''.join([format(ord(char), '08b') for char in message])

def binary_to_message(binary_str):
    return ''.join([chr(int(binary_str[i:i+8], 2)) for i in range(0, len(binary_str), 8)])

def message_to_bin(message):
    binary_message = ''.join(format(ord(i), '08b') for i in message)
    return binary_message

def bin_to_message(binary_message):
    binary_values = [binary_message[i:i+8] for i in range(0, len(binary_message), 8)]
    ascii_characters = [chr(int(bv, 2)) for bv in binary_values]
    decoded_message = ''.join(ascii_characters)
    return decoded_message

def encode_message_into_weights(model, message):
    binary_secret_msg = message_to_bin(secret_message) + '1111111111111110'  # 16-bit delimiter which indicates end of the secret message
    #weights = model.get_weights()
    #print(len(model.layers))

    data_index = 0
    for i in range(len(model.layers)):
      weights = model.layers[i].get_weights()[0]     #weights
      biases = model.layers[i].get_weights()[1]        #bias

      for j, row in enumerate(range(weights.shape[0])):
        for k, col in enumerate(range(weights.shape[1])):
          if data_index < len(binary_secret_msg):
            binary_representation = float_to_binary(weights[j,k])[:-1]+binary_secret_msg[data_index]
            data_index += 1
            converted_float = binary_to_float(binary_representation)
            weights[j,k] = converted_float
      model.layers[i].set_weights([weights, biases])


def decode_message_from_weights(model):
    binary_secret_msg = ""
    weights = model.get_weights()

    for i in range(len(model.layers)):
      weights = model.layers[i].get_weights()[0]     #weights
      for j, row in enumerate(range(weights.shape[0])):
        for k, col in enumerate(range(weights.shape[1])):
          binary_secret_msg += (float_to_binary(weights[j,k])[-1])

    end_delimiter = '1111111111111110'
    end_delimiter_pos = binary_secret_msg.find(end_delimiter)
    if end_delimiter_pos != -1:
        binary_secret_msg = binary_secret_msg[:end_delimiter_pos]

    decoded_message = bin_to_message(binary_secret_msg)
    return decoded_message


if __name__ == '__main__':

    # Secret message you want to hide
    secret_message = "Good morning Qatar"
    encode_message_into_weights(model, secret_message)
    decoded_message = decode_message_from_weights(model)
    print(decoded_message)

    y_pred = np.round(model.predict(data))
    print(y_pred)
    print('Model accuracy = ', accuracy_score(labels, y_pred)*100)

Good morning Qatar
1/1 [==============================] - 0s 15ms/step
[[0.]
 [1.]
 [1.]
 [0.]]
Model accuracy =  100.0
